# Tugas 4 ( Menggabungkan Data )

In [15]:
import pyodbc 
import pandas as pd
#sql server
cnxn_str = ("Driver={odbc driver 17 for sql server};"
            "Server=LAPTOP-SUOTNT28\SQLEXPRESS;"
            "Database=petalwidth;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()	
cursor.execute("SELECT * FROM dbo.petalwidth") 
row = cursor.fetchall() 
cursor.close()
data=[]
for i in row:
     data.append([i[0],i[1],i[2]])
df_sqlServer = pd.DataFrame(data, columns=['id','petalwidth','class'])
df_sqlServer


,id,petalwidth,class
0,1,0.2,Iris-setosa
1,2,0.2,Iris-setosa
2,3,0.2,Iris-setosa
3,4,0.2,Iris-setosa
4,5,0.2,Iris-setosa
...,...,...,...
145,146,2.3,Iris-virginica
146,147,1.9,Iris-virginica
147,148,2.0,Iris-virginica
148,149,2.3,Iris-virginica


In [16]:
!pip install psycopg2

In [17]:
import psycopg2
import pandas as pd
#postgressql elephant
param_dic = {
    "host"      : "floppy.db.elephantsql.com",
    "database"  : "yreejdrp",
    "user"      : "yreejdrp",
    "password"  : "Ukeze-Xysa-i3WBUwU0Z6tcYMgNmTVuQ"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)

column_names = ['id','sepalwidth']

df_postEle = postgresql_to_dataframe(conn, "select * from sepalwidth", column_names)
df_postEle

Connecting to the PostgreSQL database...
Connection successful


,id,sepalwidth
0,1,3.5
1,2,3.0
2,3,3.2
3,4,3.1
4,5,3.6
...,...,...
145,146,3.0
146,147,2.5
147,148,3.0
148,149,3.4


In [18]:
import psycopg2
import pandas as pd
#postgres local
param_dic = {
    "host"      : "localhost",
    "database"  : "sepallength",
    "user"      : "postgres",
    "password"  : "postgres"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)

column_names = ['id','sepallength']

df_postLocal = postgresql_to_dataframe(conn, "select * from sepallength", column_names)
df_postLocal

Connecting to the PostgreSQL database...
Connection successful


,id,sepallength
0,1,5.1
1,2,4.9
2,3,4.7
3,4,4.6
4,5,5.0
...,...,...
145,146,6.7
146,147,6.3
147,148,6.5
148,149,6.2


In [19]:
!pip install mysqlclient
!pip install mysql-connector-python
!pip install pymysql

In [20]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="petallength"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM petallength")

myresult = mycursor.fetchall()
mycursor.close()
df_mysql = pd.DataFrame(myresult, columns=['id','petallength'])
# df_mysql.drop(df_mysql.columns[[0]],axis=1,inplace=True)
df_mysql

,id,petallength
0,1,1.4
1,2,1.4
2,3,1.3
3,4,1.5
4,5,1.4
...,...,...
145,146,5.2
146,147,5.0
147,148,5.2
148,149,5.4


In [24]:
# merge df
result = pd.concat([df_postLocal, df_postEle,df_mysql, df_sqlServer], axis=1)
# result = pd.merge(df_postLocal,df_sqlServer, how="inner", on=["id", "id"])
result

,id,sepallength,id,sepalwidth,id,petallength,id,petalwidth,class
0,1,5.1,1,3.5,1,1.4,1,0.2,Iris-setosa
1,2,4.9,2,3.0,2,1.4,2,0.2,Iris-setosa
2,3,4.7,3,3.2,3,1.3,3,0.2,Iris-setosa
3,4,4.6,4,3.1,4,1.5,4,0.2,Iris-setosa
4,5,5.0,5,3.6,5,1.4,5,0.2,Iris-setosa
...,...,...,...,...,...,...,...,...,...
145,146,6.7,146,3.0,146,5.2,146,2.3,Iris-virginica
146,147,6.3,147,2.5,147,5.0,147,1.9,Iris-virginica
147,148,6.5,148,3.0,148,5.2,148,2.0,Iris-virginica
148,149,6.2,149,3.4,149,5.4,149,2.3,Iris-virginica


In [25]:
result.drop(result.columns[[0,2,4,6]],axis=1,inplace=True)
result

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [26]:
X = result.values[:, :4]
Y = result.values[:, 4]
print(X)

[[Decimal('5.1') Decimal('3.5') Decimal('1.4') Decimal('0.2')]
 [Decimal('4.9') Decimal('3.0') Decimal('1.4') Decimal('0.2')]
 [Decimal('4.7') Decimal('3.2') Decimal('1.3') Decimal('0.2')]
 [Decimal('4.6') Decimal('3.1') Decimal('1.5') Decimal('0.2')]
 [Decimal('5.0') Decimal('3.6') Decimal('1.4') Decimal('0.2')]
 [Decimal('5.4') Decimal('3.9') Decimal('1.7') Decimal('0.4')]
 [Decimal('4.6') Decimal('3.4') Decimal('1.4') Decimal('0.3')]
 [Decimal('5.0') Decimal('3.4') Decimal('1.5') Decimal('0.2')]
 [Decimal('4.4') Decimal('2.9') Decimal('1.4') Decimal('0.2')]
 [Decimal('4.9') Decimal('3.1') Decimal('1.5') Decimal('0.1')]
 [Decimal('5.4') Decimal('3.7') Decimal('1.5') Decimal('0.2')]
 [Decimal('4.8') Decimal('3.4') Decimal('1.6') Decimal('0.2')]
 [Decimal('4.8') Decimal('3.0') Decimal('1.4') Decimal('0.1')]
 [Decimal('4.3') Decimal('3.0') Decimal('1.1') Decimal('0.1')]
 [Decimal('5.8') Decimal('4.0') Decimal('1.2') Decimal('0.2')]
 [Decimal('5.7') Decimal('4.4') Decimal('1.5') Decimal(

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

In [28]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [29]:
y_pred = classifier.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score
print (" DecisionTree Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 DecisionTree Accuracy :  96.66666666666667


In [31]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [32]:
y_pred = classifier.predict(X_test)

In [33]:
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 KNN Accuracy :  100.0
